In [ ]:
import sys
import os

# Define the path to your folder
folder_path = r"C:\Users\Tawfique\Thesis\Data_Augmentation\Models\AutoDiffusion"

# Add the folder to sys.path
sys.path.append(folder_path)

In [10]:
import numpy as np
#import process_edited as pce
import process_GQ as pce
import autoencoder as ae
import diffusion as diff
import TabDDPMdiff as TabDiff
import pandas as pd
import torch
import os
import time
import numpy as np

In [ ]:
# strings_set = {'abalone', 'adult', 'Churn_Modelling','faults', 'indian_liver_patient', 
#                'insurance', 'News', 'Obesity', 'Shoppers', 'Titanic', 'wilt', 'Bean', 'nursery', 'Magic', 'HTRU'}
Model = 'AutoDiff'

dataset='diabetes'
file_path = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
# Read dataframe
# print(file_path)
real_df = pd.read_csv(file_path)
#real_df = real_df.drop('url', axis=1)
# # Step 2: Inspect the data and check for class imbalance
# # Assuming the last column is the label, and the rest are features
X = real_df.iloc[:, :-1].values  # Features
y = real_df.iloc[:, -1].values  # Labels (binary classification)
#  # Separate the minority class
real_minortiy = real_df[y == 1]

    

C:\Users\Tawfique\Thesis\Data_Augmentation\Datasets\oil.csv


41

In [26]:
print('Real data shape:', real_df.shape)


Real data shape: (937, 50)


In [12]:
threshold = 0.01 # Threshold for mixed-type variables
parser = pce.DataFrameParser().fit(real_minortiy, threshold)
################################################################################################################
# Auto-encoder hyper-parameters
device = 'cuda' #@param {'type':'string'}
n_epochs = 2000 #@param {'type':'integer'}
eps = 1e-5 #@param {type:"number"}
weight_decay = 1e-6 #@param {'type':'number'}
maximum_learning_rate = 1e-2 #@param {'type':'number'}
lr = 2e-4 #@param {'type':'number'}
hidden_size = 250
num_layers = 3
batch_size = real_minortiy.shape[0] # Full batch

ds = ae.train_autoencoder(real_minortiy, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
latent_features = ds[1].detach()
    

  0%|          | 0/2000 [00:00<?, ?it/s]

In [13]:
from ForestDiffusion import ForestDiffusionModel
import torch
import numpy as np
import pandas as pd

# Convert to NumPy array
array = latent_features.detach().cpu().numpy()
array.shape


(41, 69)

In [14]:
import pandas as pd
df1=pd.DataFrame(array)

In [15]:
forest_model = ForestDiffusionModel(array, label_y=None, n_t=50, duplicate_K=100, bin_indexes=[], cat_indexes=[], int_indexes=[], diffusion_type='flow', n_jobs=-1)

In [30]:
minority_fake = forest_model.generate(batch_size=len(real_minortiy)) # Adjust the batch size to create a balanced dataset
sample=torch.tensor(minority_fake, dtype=torch.float32)
sample.shape

torch.Size([41, 69])

In [ ]:
gen_output = ds[0](sample, ds[2], ds[3])
gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)

output_directory =  f'..\\..\\..\\Datasets\\Synthetic Data\\'
filename = f'{Model}+Forest_{dataset}_Synthetic.csv'
output_file = os.path.join(output_directory, filename)
gen_df.to_csv(output_file, index=False) 


# Select a random sample of the generated data
selected_samples = gen_df.sample(n=min(100,gen_df.shape[0]), random_state=42)  # For reproducibility
# Syn _df will be the dataset after augmentation
syn_df = pd.concat([real_df, selected_samples], ignore_index=True)


augmented_output_directory =  f'..\\..\\..\\Datasets\\Augmented Data\\'
filename = f'{Model}+Forest_{dataset}_Augmented.csv'
augmented_output_file = os.path.join(augmented_output_directory, filename)
syn_df.to_csv(augmented_output_file, index=False) 

In [18]:
import os
Model='Autodiff'
dataset='diabetes'
augmented_output_directory =  f'..\\..\\..\\Datasets\\Augmented Data\\'
filename = f'{Model}+Forest_{dataset}_Augmented.csv'
augmented_output_file = os.path.join(augmented_output_directory, filename)
print(f'{augmented_output_file}')

..\..\..\Datasets\Augmented Data\Autodiff+Forest_diabetes_Augmented.csv


In [25]:
syn_df.shape

(978, 50)

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from ForestDiffusion import ForestDiffusionModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, f1_score

# real_df = pd.read_csv(filename)
# syn_filename = f'{string}/{Model}_{string}_Augmented.csv'

# augmented_df = pd.read_csv(syn_filename)
augmented_df=syn_df

X = real_df.iloc[:, :-1].values  # Features
y = real_df.iloc[:, -1].values 
# Check and print the original class distribution
unique, counts = np.unique(y, return_counts=True)
class_dist_before = dict(zip(unique, counts))
print(f"Class distribution before augmentation: {class_dist_before}")# Labels (binary classification)

X_balanced = augmented_df.iloc[:, :-1].values  # Features
y_balanced = augmented_df.iloc[:, -1].values  # Labels (binary classification)

# Check and print the Augmented class distribution
unique, counts = np.unique(y_balanced, return_counts=True)
class_dist_after = dict(zip(unique, counts))
print(f"Class distribution after augmentation: {class_dist_after}")

# Step 6: Split the dataset into training and test sets (original and balanced)
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# Step 7: Train a simple classifier on both original and generated datasets
clf_orig = RandomForestClassifier(random_state=42)
clf_orig.fit(X_train_orig, y_train_orig)

clf_bal = RandomForestClassifier(random_state=42)
clf_bal.fit(X_train_bal, y_train_bal)

# Step 8: Predict and calculate recall and F1 scores
y_pred_orig = clf_orig.predict(X_test_orig)
y_pred_bal = clf_bal.predict(X_test_orig)

recall_orig = recall_score(y_test_orig, y_pred_orig)
recalls_bal = recall_score(y_test_orig, y_pred_bal)

f1_orig = f1_score(y_test_orig, y_pred_orig)
f1_bal = f1_score(y_test_orig, y_pred_bal)

# Step 9: Print the performance metrics
print(f"Recall score (original data): {recall_orig:.4f}")
print(f"Recall score (generated data): {recalls_bal:.4f}")
print(f"F1 score (original data): {f1_orig:.4f}")
print(f"F1 score (generated data): {f1_bal:.4f}")
print("Classification Report (original data):\n", classification_report(y_test_orig, y_pred_orig))
print("Classification Report (generated data):\n", classification_report(y_test_orig, y_pred_bal))


print(f"Number of fake samples generated: {len(augmented_df)-len(real_df)}")

Class distribution before augmentation: {-1: 896, 1: 41}
Class distribution after augmentation: {-1.0: 896, 1.0: 82}
Recall score (original data): 0.2727
Recall score (generated data): 0.6364
F1 score (original data): 0.3750
F1 score (generated data): 0.7778
Classification Report (original data):
               precision    recall  f1-score   support

          -1       0.97      0.99      0.98       271
           1       0.60      0.27      0.38        11

    accuracy                           0.96       282
   macro avg       0.79      0.63      0.68       282
weighted avg       0.96      0.96      0.96       282

Classification Report (generated data):
               precision    recall  f1-score   support

          -1       0.99      1.00      0.99       271
           1       1.00      0.64      0.78        11

    accuracy                           0.99       282
   macro avg       0.99      0.82      0.89       282
weighted avg       0.99      0.99      0.98       282

Number